## Vertex AI OCR pretrained model
This notebook demonstrate how you can extract text from image file.

In [ ]:
import io
from google.cloud import vision
import google.auth
from google.auth.transport import requests
from google.api_core.client_options import ClientOptions
import requests as reqs

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "<your service account key file>" 
os.environ["GRPC_DEFAULT_SSL_ROOTS_FILE_PATH"] = "<your trusted bundle certs>"

In [ ]:
audience = "https://<vertex ai ocr endpoint>:443" #e.g. vision.org-1.acme.com
api_endpoint = "<vertex ai ocr endpoint>"
project_id = "<your project id>"

In [4]:
# Get your credentials
def get_credentials():
    creds = None
    try:
        creds, project_id = google.auth.default()
        creds = creds.with_gdch_audience(audience)
        sesh = reqs.Session()
        sesh.verify = False
        creds._ca_cert_path = False
        req = requests.Request(session=sesh)
        creds.refresh(req)
        print("Got token: ")
        print(creds.token)
    except Exception as e:
        print("Caught exception" + str(e))
        raise e
    return creds

In [5]:
def vision_client(creds):
    opts = ClientOptions(api_endpoint=api_endpoint)
    """Create vision client."""
    return vision.ImageAnnotatorClient(credentials=creds, client_options=opts)

In [ ]:
def detect_text(image_file_path):
    creds = get_credentials()
    vc = vision_client(creds)

    metadata = [("x-goog-user-project", "projects/" + project_id)]

    with io.open(image_file_path, 'rb') as image_file:
        content = image_file.read()

    image = vision.Image(content=content)

    response = vc.text_detection(image=image, metadata=metadata)
    texts = response.text_annotations
    print('Texts:')

    for text in texts:
        print('\n"{}"'.format(text.description))
        vertices = (['({},{})'.format(vertex.x, vertex.y)
        for vertex in text.bounding_poly.vertices])
        print('bounds: {}'.format(','.join(vertices)))

    # Add error handling code here.
    if response.error.message:
        raise Exception(response.error.message)

In [ ]:
detect_text("sign_text.png")